In [ ]:
from typing import Dict, List
from utils import multiply_bit_string_with_matrix
import json
import shutil

In [ ]:
def process_matrix_string(inp: str) -> List[List[int]]:
    """Takes as input a matrix without commas (from SageMath), and returns the Python matrix."""
    i = 0
    built = ""
    while i < len(inp):
        built += inp[i]
        if (inp[i] == "1" or inp[i] == "0") and (i < len(inp) and inp[i+1] != "]"):
            built += ","
        if inp[i] == "]" and not i == len(inp) - 2:
            built += ","
        i += 1
    return eval(f"[{built}]".replace("\n", ""))

In [ ]:
parity_check_matrix = process_matrix_string('''
[1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
[0 1 1 0 0 1 1 0 0 1 1 0 0 1 1]
[0 0 0 1 1 1 1 0 0 0 0 1 1 1 1]
[0 0 0 0 0 0 0 1 1 1 1 1 1 1 1]
''')

In [ ]:
def kbits(n, k):
    # From https://stackoverflow.com/questions/58069431/find-all-binary-strings-of-certain-weight-has-fast-as-possible
    limit = 1 << n
    val = (1 << k)-1
    while val < limit:
        yield "{0:0{1}b}".format(val, n)
        minbit = val & -val  # rightmost 1 bit
        fillbit = (val+minbit) & ~val  # rightmost 0 to the left of that bit
        val = val+minbit | (fillbit//(minbit << 1))-1


def build_decoding_lookup_table(codeword_length: int, max_error_string_weight: int) -> Dict[str, str]:
    table = {}
    table["0"*len(parity_check_matrix)] = "0" * codeword_length
    for weight in range(1, max_error_string_weight + 1):
        for error_string in kbits(codeword_length, weight):
            xH_transpose = multiply_bit_string_with_matrix(
                error_string, H_transpose)
            table[xH_transpose] = error_string
    return table

In [ ]:
H_transpose = [[row[j] for row in parity_check_matrix]
               for j in range(len(parity_check_matrix[0]))]
decoding_lookup_table = build_decoding_lookup_table(
    len(parity_check_matrix[0]), 1)

In [ ]:
code_name = "hamming_4"

with open(f"error_correcting_codes/{code_name}_parity_check_matrix.txt", "w") as f:
    json.dump(parity_check_matrix, f)

with open(f"error_correcting_codes/{code_name}_table.txt", "w") as f:
    json.dump(decoding_lookup_table, f)

shutil.make_archive("error_correcting_codes", "zip", "error_correcting_codes");